## EDA running notebook

In [6]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [3]:
# set up connecting to the met database
conn = sqlite3.connect("../data/met.db")
cursor = conn.cursor()

In [9]:
# pull categorical data types for further visualization
categorical = ['isHighlight', 'isPublicDomain', 'country', 'classification']

# pull in department names for future visualizations
department_names = pd.read_sql("SELECT display_name FROM Department", conn)

In [4]:
def one_p_filter(pie_df, field):
    '''collapses the entries in the given field to Other for entries under 1%'''
    if type(pie_df.dtypes[field]) is str or type(pie_df.dtypes[field]) == np.dtypes.ObjectDType:
        cutoff = sum(pie_df['num_objects']) * 0.01
        pie_df.loc[pie_df['num_objects'] < cutoff, field] = "Other " + field
    return pie_df.copy()

In [8]:
def select_dept_field(dept, field, filter_1_p):
    '''filters the met db by the department and groups by field. can optionally roll up groups with under 1% of the total count'''
    pie_data = pd.read_sql(f'''SELECT a.{field}, COUNT(*) as num_objects 
                           FROM Art a, Objects o, Department d WHERE d.display_name="''' + dept +  f'''" AND a.object_id=o.object_id 
                           AND o.department_id=d.department_id GROUP BY {field}''', conn)

    if field in ("isHighlight", "isPublicDomain"):
        pie_data[field] = pie_data[field].map({0:"No", 1:"Yes"})
    elif filter_1_p:
        pie_data= one_p_filter(pie_data, field)
    
    fig = px.pie(pie_data, values='num_objects', names=field, title=f'{dept} Department {field} Breakdown')
    fig.show()

In [14]:
# create the interactive pie chart
interactive_plot = widgets.interactive(select_dept_field, 
    dept= widgets.Dropdown(
        options=department_names['display_name'].to_list(),
        value='The Cloisters',
        description='Department:',
        disabled=False), 
    field=widgets.ToggleButtons(
        options=categorical,
        description='Field of Interest:',
        disabled=False),
    filter_1_p= widgets.Checkbox(
        value=False,
        description='Collapse values under 1%?',
        disabled=False,
        button_style='success',
        tooltip='Description',
        icon='check' #
    ))

In [15]:
interactive_plot

interactive(children=(Dropdown(description='Department:', index=5, options=('American Decorative Arts', 'Ancie…

In [16]:
conn.close()